Una volta definito il corpus, esso viene sottoposto a un processo di indicizzazione basato sugli strumenti classici dell’Information Retrieval, come TF-IDF o BM25. La pipeline prevede la tokenizzazione del testo, la rimozione delle stopword, la normalizzazione linguistica, la creazione del vocabolario e infine la generazione delle matrici vettoriali che rappresentano ogni documento. L’indice risultante è persistente, efficiente e pronto per essere interrogato da qualsiasi query dello scout.

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text
import joblib

# ============================
# 1. Config e caricamento dati
# ============================

CORPUS_JSON_PATH = "../data/processed/nba_players_corpus.json"
MODELS_DIR = "../data/models"

os.makedirs(MODELS_DIR, exist_ok=True)

# Carica corpus
corpus_df = pd.read_json(CORPUS_JSON_PATH)
corpus_df["text_profile"] = corpus_df["text_profile"].fillna("")

print("Corpus shape:", corpus_df.shape)

# Verifica che la colonna Status esista
status_available = "Status" in corpus_df.columns

# ============================
# 2. Aggiunta stato al testo
# ============================

if status_available:
    corpus_df["status_text"] = corpus_df["Status"].replace({
        "Active": "giocatore attivo",
        "Retired": "giocatore ritirato"
    })

    # arricchiamo il testo rendendo lo stato ricercabile
    corpus_df["full_text"] = corpus_df["text_profile"] + " " + corpus_df["status_text"]
else:
    corpus_df["full_text"] = corpus_df["text_profile"]

# ============================
# 3. Stopwords e TF-IDF
# ============================

# Stopwords personalizzate italiane
stopwords_it = [
    "giocatore", "giocatori", "ruolo", "provenienza",
    "nba", "draft", "stagioni", "partita", "partite",
]

# Trasformiamo tutto in LISTA (come richiesto da sklearn)
STOPWORDS = list(text.ENGLISH_STOP_WORDS) + stopwords_it

vectorizer = TfidfVectorizer(
    lowercase=True,
    ngram_range=(1, 2),
    max_df=0.90,        
    min_df=1,            
    stop_words=STOPWORDS
)


tfidf_matrix = vectorizer.fit_transform(corpus_df["full_text"])
print("TF-IDF matrix shape:", tfidf_matrix.shape)


# ============================
# 4. Salvataggio index IR
# ============================

VECTORIZER_PATH = os.path.join(MODELS_DIR, "tfidf_vectorizer.joblib")
MATRIX_PATH = os.path.join(MODELS_DIR, "tfidf_matrix.joblib")
META_PATH = os.path.join(MODELS_DIR, "index_metadata.csv")

joblib.dump(vectorizer, VECTORIZER_PATH)
joblib.dump(tfidf_matrix, MATRIX_PATH)

meta_cols = [c for c in [
    "player_id", "Player", "DraftYear", "Pick", 
    "PickBand", "Status", "text_profile"
] if c in corpus_df.columns]

metadata_df = corpus_df[meta_cols].reset_index(drop=True)
metadata_df.to_csv(META_PATH, index=False)

print("Indice salvato in:")
print(" -", VECTORIZER_PATH)
print(" -", MATRIX_PATH)
print(" -", META_PATH)

# ============================
# 5. Funzione di ricerca
# ============================

def normalize_query(query):
    """Normalizza la query utente: sinonimi, forme diverse, stato, ruoli."""
    q = query.lower().strip()

    # --- Stato ---
    q = q.replace("in attività", "giocatore attivo")
    q = q.replace("attivo", "giocatore attivo")

    # --- Ruoli ---
    q = q.replace("guardia tiratrice", "shooting guard")
    q = q.replace("guardia", "guard")
    q = q.replace("ala grande", "power forward")
    q = q.replace("ala piccola", "small forward")
    q = q.replace("centro", "center")

    # --- Sinonimi di tiro ---
    q = q.replace("tiratrice", "tiratore")
    q = q.replace("ottimo da tre", "tiratore da 3")
    q = q.replace("forte da tre", "tiratore da 3")

    # --- Sinonimi difesa ---
    q = q.replace("forte in difesa", "buon difensore")
    q = q.replace("difensore forte", "buon difensore")

    # --- Sinonimi rimbalzo ---
    q = q.replace("forte a rimbalzo", "ottimo rimbalzista")
    q = q.replace("rimbalzi", "rimbalzista")

    return q



def search_players(query, top_k=10, active_only=False, retired_only=False):
    """
    Esegue ricerca TF-IDF + filtri:
    - active_only = restituisce solo giocatori in attività
    - retired_only = restituisce solo giocatori ritirati
    """
    
    query_clean = normalize_query(query)

    # vettorizza query
    query_vec = vectorizer.transform([query_clean])

    # similarità coseno
    sim_scores = cosine_similarity(query_vec, tfidf_matrix).ravel()

    # ordina
    order = np.argsort(sim_scores)[::-1]
    results = metadata_df.iloc[order].copy()
    results["similarity"] = sim_scores[order]
    results["score"] = (results["similarity"] * 100).round(1)

    # === FILTRI PER STATO ===
    if active_only:
        results = results[results["Status"] == "Active"]
    if retired_only:
        results = results[results["Status"] == "Retired"]

    return results.head(top_k)



Corpus shape: (8323, 7)
TF-IDF matrix shape: (8323, 22019)
Indice salvato in:
 - ../data/models\tfidf_vectorizer.joblib
 - ../data/models\tfidf_matrix.joblib
 - ../data/models\index_metadata.csv


In [2]:
# ============================
# TEST
# ============================

query = "cerco una guardia tiratrice in attività"
results = search_players(query, top_k=5, active_only=True)

pd.set_option("display.max_colwidth", 200)

cols = [c for c in ["Player", "Status", "DraftYear", "Pick", "score", "text_profile"] if c in results.columns]

print("Query:", query)
display(results[cols])


Query: cerco una guardia tiratrice in attività


,Player,Status,DraftYear,Pick,score,text_profile
8312,Harrison Ingram,Active,2024,48,14.2,Giocatore: Harrison Ingram. Proveniente da UNC. Selezionato al Draft NBA 2024 con la pick 48 (SecondRound). Giocatore attualmente in attività NBA. Segna 0.7 punti a partita. giocatore non focalizz...
8318,Anton Watson,Active,2024,54,13.9,Giocatore: Anton Watson. Proveniente da Gonzaga. Selezionato al Draft NBA 2024 con la pick 54 (SecondRound). Giocatore attualmente in attività NBA. Segna 1.2 punti a partita. giocatore non focaliz...
8286,DaRon Holmes,Active,2024,22,13.9,Giocatore: DaRon Holmes. Proveniente da Dayton. Selezionato al Draft NBA 2024 con la pick 22 (FirstRound). Giocatore attualmente in attività NBA. Segna 0.0 punti a partita. giocatore non focalizza...
8315,Melvin Ajinça,Active,2024,51,13.6,Giocatore: Melvin Ajinça. Proveniente da Overseas (France). Selezionato al Draft NBA 2024 con la pick 51 (SecondRound). Giocatore attualmente in attività NBA. Segna 0.0 punti a partita. giocatore ...
8320,Kevin McCullar Jr.,Active,2024,56,13.5,Giocatore: Kevin McCullar Jr.. Proveniente da Kansas. Selezionato al Draft NBA 2024 con la pick 56 (SecondRound). Giocatore attualmente in attività NBA. Segna 0.0 punti a partita. giocatore non fo...
